## Play around datasets and create one with hierarchical prior


In [2]:
import os
import sys
from tensorflow.compat.v1 import gfile

sys.path.append(os.path.abspath("/home/mwu34/disentanglement_lib"))
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999

DSPRITES_PATH = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites",
    "dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz"
)

In [4]:
# trying to get mapping from specific parameter combinations to model num

# from disentanglement_lib.config.unsupervised_study_v1.sweep import get_config
# from disentanglement_lib.config.double_descent_study_v10.sweep import get_config
from disentanglement_lib.config.active_learning_study_v16.sweep import get_config
# from disentanglement_lib.config.embeddings_study_v4.sweep import get_config



import pandas as pd

configs = get_config()
df = pd.DataFrame(configs)

# Get the beta_tc_vae models using dsprites and a beta param of 4.
# model = df['model.name'] == "beta_tc_vae"
# dataset = df['dataset.name'] == "dsprites_full"

# beta = df["vae.beta"] == 1.0

# Give all the possible model configs unused parameters excluded (e.g., annealed_vae.gamma)
# print(df[model & dataset & beta].dropna(axis=1))

# Give the list of model ids with this configuration
df.head(200)


,dataset.name,s2_vae.factor_sizes,s2_factor_vae.factor_sizes,s2_dip_vae.factor_sizes,s2_beta_tc_vae.factor_sizes,encoder.encoder_fn,decoder.decoder_fn,model.name,s2_vae.beta,s2_vae.gamma_sup,s2_vae.num_stochastic_passes,model.model,model.supervised_sampling_method,model.uncertainty_method,supervised_loss.loss_fn,supervised_loss.cov_loss_ratio,model.model_seed
0,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,1.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,0.000,0
1,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,1.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,0.125,0
2,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,1.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,0.500,0
3,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,1.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,1.000,0
4,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,1.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,4.000,0
5,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,4.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,0.000,0
6,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,4.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,0.125,0
7,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,4.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,0.500,0
8,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,4.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,1.000,0
9,dsprites_full,"(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)","(3, 6, 40, 32, 32)",@conv_encoder,@deconv_decoder,s2_beta_vae,4.0,1.0,1,@s2_vae(),random,dropout_mean,xent_and_cov,4.000,0


In [2]:
from disentanglement_lib.data.ground_truth import named_data
dsprites = named_data.get_named_ground_truth_data('dsprites_full', 0)

In [3]:
# debugging how to parse config files

from disentanglement_lib.methods.semi_supervised import semi_supervised_utils  # pylint: disable=unused-import
from disentanglement_lib.methods.semi_supervised import train_semi_supervised_lib
from disentanglement_lib.config.active_learning_study_v1.sweep import ActiveLearningStudyV1
from absl import logging
import gin.tf

study = ActiveLearningStudyV1()
output_directory = '/home/mwu34/disentanglement_lib/test'

model_bindings, model_config_file = study.get_model_config(0)
logging.info("Training model...")
model_dir = os.path.join(output_directory, "model")
model_bindings = [
    "model.model_num = {}".format(0),
] + model_bindings
print(model_config_file)
print(model_bindings)
gin.parse_config_files_and_bindings([model_config_file], [])

# train_semi_supervised_lib.train_with_gin(model_dir, False, [model_config_file],
#                      model_bindings)

/home/mwu34/disentanglement_lib/disentanglement_lib/config/active_learning_study_v1/model_configs/shared.gin
['model.model_num = 0', "dataset.name = 'dsprites_full'", 'encoder.encoder_fn = @conv_encoder', 'decoder.decoder_fn = @deconv_decoder', "model.name = 's2_beta_vae'", 's2_vae.beta = 1.0', 's2_vae.gamma_sup = 1.0', 'model.model = @s2_vae()', 'model.model_seed = 0']


[ParsedConfigFileIncludesAndImports(filename='/home/mwu34/disentanglement_lib/disentanglement_lib/config/active_learning_study_v1/model_configs/shared.gin', imports=[], includes=[])]

In [4]:
from disentanglement_lib.methods.semi_supervised import semi_supervised_utils  # pylint: disable=unused-import
from disentanglement_lib.methods.semi_supervised import train_semi_supervised_lib

# (sampled_observations, sampled_factors, factor_sizes) = semi_supervised_utils.sample_supervised_data(
#        0, dsprites, 100
# )
(sampled_observations, sampled_factors, factor_sizes) = semi_supervised_utils.load_supervised_data(
       0, dsprites, 1000, "highest_summed_std"
)

(1000, 64, 64, 1)


In [7]:
from disentanglement_lib.data.ground_truth import named_data


dsprites_subset = named_data.get_named_ground_truth_data(
    'dsprites_full',
    train_with_full_dataset=False,
    split_method='train',
    num_training_data=700000
)

In [11]:
random_state = np.random.RandomState(0)
factors, images = dsprites_subset.sample(5, random_state)

AttributeError: 'SubsetDSprites' object has no attribute 'dataset_indices'

In [42]:
num_factors = 5

def index_to_factors(index_array, factor_bases):
    factor_bases = factor_bases.astype(int)
    print(factor_bases)
    factors = np.zeros((len(index_array), num_factors))
    for factor_idx, factor_base in enumerate(factor_bases[1:]):
        factors[:, factor_idx], index_array = np.divmod(index_array, factor_base)
    return factors

def factors_to_index(factor_array, factor_bases):
    indices = np.array(np.dot(factor_array, factor_bases), dtype=np.int64)
    return indices

In [40]:
gg = np.array([10000, 20000, 30000, 700000])
res = index_to_factors(gg, dsprites.factor_bases)

[737280 245760  40960   1024     32      1]


In [45]:
back = factors_to_index(res, dsprites.factor_bases[1:])
back

array([ 10000,  20000,  30000, 700000])

In [32]:
dsprites.factor_bases

array([7.3728e+05, 2.4576e+05, 4.0960e+04, 1.0240e+03, 3.2000e+01,
       1.0000e+00])

In [7]:

a, b = dsprites.sample(10, random_state)

In [10]:
a.shape

(10, 5)

In [17]:
# split dsprites dataset into two parts

full_size = np.prod(dsprites.factor_sizes)
train_dataset_size = 7000
train_indices = np.random.choice(full_size, train_dataset_size, replace=False)
train_indices = np.sort(train_indices)
valid_indices = np.delete(np.arange(full_size), train_indices)

train_indices_path = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites", f"train{train_dataset_size}.npy"
)
valid_indices_path = os.path.join(
    os.environ.get("DISENTANGLEMENT_LIB_DATA", "."), "dsprites", f"valid{train_dataset_size}.npy"
)
np.save(train_indices_path, train_indices)
np.save(valid_indices_path, valid_indices)

In [21]:
gg = np.random.choice([10, 50, 3], 3, replace=True)
gg

array([ 3, 50, 50])

In [62]:
4688 in valid_indices

True

In [23]:
len(gg)

3